# Reading Avro Blobs Into Parquet Data Sets

In [1]:
import os
import string
import json
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession, SQLContext, Window
from pyspark.sql.functions import udf, mean, lit, stddev, col, expr, when, date_sub, avg, window
from pyspark.sql.types import DoubleType, ArrayType, ShortType, LongType, IntegerType, TimestampType, StructType, StringType, StructField
from pyspark.storagelevel import StorageLevel
from azureml.api.schema.dataTypes import DataTypes
from azureml.api.schema.sampleDefinition import SampleDefinition
from azureml.api.realtime.services import generate_schema

STORAGE_ACCOUNT_SUFFIX = 'core.windows.net'
TELEMETRY_STORAGE_ACCOUNT_NAME = "stguwsjc4vojbrbw"
TELEMETRY_STORAGE_ACCOUNT_KEY = "2GGDWVBtGBy+hAgl5a1uGT4NeU2zzFdocuDFKnOwR2vc5wEOP7jTfbS3/Nl5vvzEudJ4nfH6ozmSOSPXo3xETA=="
TELEMETRY_CONTAINER_NAME = "telemetry"
LOGS_ARCHIVE_CONTAINER_NAME = 'logs-archive'
STAGING_STORAGE_ACCOUNT_NAME = "stguwsjc4vojbrbw"
STAGING_STORAGE_ACCOUNT_KEY = "2GGDWVBtGBy+hAgl5a1uGT4NeU2zzFdocuDFKnOwR2vc5wEOP7jTfbS3/Nl5vvzEudJ4nfH6ozmSOSPXo3xETA=="


In [2]:
from pathlib import Path
data_dir = str(Path.home()) + '/data'

% rm -rf $data_dir
% mkdir $data_dir $data_dir/logs $data_dir/telemetry

In [8]:
wasbTelemetryUrl = "wasb://{0}@{1}.blob.{2}/*/*/*/*/*/*/*".format(TELEMETRY_CONTAINER_NAME, 
                                                                  TELEMETRY_STORAGE_ACCOUNT_NAME, 
                                                                  STORAGE_ACCOUNT_SUFFIX)
sc = SparkSession.builder.getOrCreate()
hc = sc._jsc.hadoopConfiguration()
hc.set("avro.mapred.ignore.inputs.without.extension", "false")
if TELEMETRY_STORAGE_ACCOUNT_KEY:
     hc.set("fs.azure.account.key.{}.blob.core.windows.net".format(TELEMETRY_STORAGE_ACCOUNT_NAME), TELEMETRY_STORAGE_ACCOUNT_KEY)
hc.set("fs.azure.account.key.{}.blob.core.windows.net"
    .format(STAGING_STORAGE_ACCOUNT_NAME), STAGING_STORAGE_ACCOUNT_KEY)
sql = SQLContext.getOrCreate(sc)
avroblob = sql.read.format("com.databricks.spark.avro").load(wasbTelemetryUrl)
avroblob.show()

+--------------------+----------+--------------------+--------------------+
|     EnqueuedTimeUtc|Properties|    SystemProperties|                Body|
+--------------------+----------+--------------------+--------------------+
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T

In [38]:
#get telemetry data from avro blob (contained in 'body' column)
pandas_avroblob = avroblob.toPandas()
pandas_avroblob_body = pandas_avroblob['Body']
#convert from binary data to string format
avroblob_tostring = pandas_avroblob_body.apply(lambda x: x.decode("utf-8"))
avroblob_tostring.head()

0    {"machineID": "Machine-004", "timestamp": "201...
1    {"machineID": "Machine-001", "timestamp": "201...
2    {"machineID": "Machine-004", "timestamp": "201...
3    {"machineID": "Machine-001", "timestamp": "201...
4    {"machineID": "Machine-004", "timestamp": "201...
Name: Body, dtype: object

In [40]:
telemetry_df = pd.DataFrame(avroblob_tostring)
telemetry_df.head()

,Body
0,"{""machineID"": ""Machine-004"", ""timestamp"": ""201..."
1,"{""machineID"": ""Machine-001"", ""timestamp"": ""201..."
2,"{""machineID"": ""Machine-004"", ""timestamp"": ""201..."
3,"{""machineID"": ""Machine-001"", ""timestamp"": ""201..."
4,"{""machineID"": ""Machine-004"", ""timestamp"": ""201..."


In [ ]:
telemetry_df.write.parquet('{0}/telemetry/telemetry.parquet'.format(data_dir), mode='overwrite')